# 3-Layer 인공신경망을 이용한 상변화 예측 실습
이 노트북은 Ti-Al-V 조성 데이터를 이용하여 상변화(고상, 액상, 다상)를 예측하는 분류 모델을 PyTorch 기반 인공신경망으로 학습합니다.

In [ ]:
# 기본 패키지 import
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
# 데이터 로딩
url = 'https://raw.githubusercontent.com/ginktepal/phase-prediction-ml/main/composition_phase_data.csv'
df = pd.read_csv(url)
df.head()

In [ ]:
# 전처리
X = df[['Ti', 'Al', 'V']].values
y = df['Phase'].values

# 라벨 인코딩
le = LabelEncoder()
y = le.fit_transform(y)

# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 학습/검증 나누기
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# 텐서 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# 3-Layer 신경망 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(16, 3)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

In [ ]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 학습 루프
train_losses = []
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

In [ ]:
# 손실 시각화
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True)
plt.show()

In [ ]:
# 테스트 예측 및 평가
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_classes = torch.argmax(y_pred, dim=1)

print(classification_report(y_test, y_pred_classes, target_names=le.classes_))

## 📊 결과 해석

- 위의 `classification_report` 출력 결과는 각 상(Phase)에 대해 **Precision, Recall, F1-score, Support**를 보여줍니다.

### 주요 용어 설명
- **Precision (정밀도)**: 예측한 것 중 실제로 맞춘 비율  
- **Recall (재현율)**: 실제인 것 중 예측에 성공한 비율  
- **F1-score**: Precision과 Recall의 조화 평균  
- **Support**: 각 클래스에 속한 샘플 개수

---

### ✅ 예시 해석 (예시 값)

| Phase | Precision | Recall | F1-score | Support |
|-------|-----------|--------|----------|---------|
| 고상  | 0.88      | 0.85   | 0.86     | 60      |
| 액상  | 0.81      | 0.90   | 0.85     | 58      |
| 다상  | 0.79      | 0.72   | 0.75     | 42      |

- **고상**은 높은 Precision과 Recall을 모두 갖추어 예측이 안정적입니다.
- **액상**은 Recall이 특히 높아서 실제 액상을 잘 맞추는 편입니다.
- **다상**은 상대적으로 어려운 클래스이며 F1-score가 가장 낮을 수 있습니다.

---

### 📌 참고
- `다상`처럼 복잡한 클래스는 더 많은 데이터나 모델 구조 개선을 통해 성능을 향상시킬 수 있습니다.
- 학습 손실(Loss) 그래프와 함께 보면서 과적합 여부도 체크해보세요.


## 🔍 예측 결과 시각화 및 모델 성능 비교

이제 Confusion Matrix를 통해 예측 정확도를 시각적으로 확인할 수 있으며,  
기존 머신러닝(RandomForestClassifier)과 현재의 인공신경망(NN) 결과를 나란히 비교할 수 있습니다.

- Confusion Matrix는 모델이 어떤 클래스를 얼마나 잘 맞췄는지를 보여줍니다.
- classification_report로 precision, recall, f1-score를 수치로 비교합니다.

이 과정을 통해 모델별 강점과 약점을 파악하고, 어떤 방향으로 개선할지 아이디어를 얻을 수 있습니다.


In [ ]:
# Confusion Matrix 시각화
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix (Neural Network)")
plt.show()


In [ ]:
# RandomForest 성능과 비교
from sklearn.ensemble import RandomForestClassifier

# 학습
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)

# Confusion Matrix
rf_cm = confusion_matrix(y_test, rf_preds)
disp_rf = ConfusionMatrixDisplay(confusion_matrix=rf_cm, display_labels=le.classes_)
disp_rf.plot(cmap=plt.cm.Oranges)
plt.title("Confusion Matrix (Random Forest)")
plt.show()

# 성능 비교 출력
print("== Neural Network ==")
print(classification_report(y_test, y_pred_classes, target_names=le.classes_))
print("== Random Forest ==")
print(classification_report(y_test, rf_preds, target_names=le.classes_))
